### 1) Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:
- InvoiceNo – номер транзакции
- StockCode – код товара
- Description – описание товара
- Quantity – количество единиц товара, добавленных в заказ
- InvoiceDate – дата транзакции 
- UnitPrice – цена за единицу товара
- CustomerID – id клиента
- Country – страна, где проживает клиент

Импортируйте данные data.csv.zip (в папке) с кодировкой ISO-8859-1. Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.

In [51]:
import pandas as pd

In [52]:
retail = pd.read_csv(r'C:\Users\arkhi\Jupyter notebook\data\data.csv.zip', encoding = 'ISO-8859-1')

In [53]:
retail_columns = retail.columns

In [54]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

### 2) Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.

In [55]:
after = retail.drop_duplicates().shape[0]

In [56]:
before = retail.shape[0]

In [57]:
before - after

5268

In [58]:
retail.drop_duplicates(inplace = True)

### 3) ⭐️Задание со звёздочкой!⭐️
Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
Сколько всего заказов отменили пользователи?

In [59]:
retail.InvoiceNo.str.startswith('C').sum()

9251

### 4) Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. 
В качестве ответа укажите число оставшихся строк.

In [60]:
retail = retail.query('Quantity > 0')

In [61]:
retail.shape[0]

526054

### 5) 💀 Сложное задание! 💀
Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id). Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

In [73]:
customer_invoices = retail.query('Country == "Germany"').groupby('CustomerID', as_index = False).agg({'InvoiceNo':'nunique'})

In [85]:
quantile_80 = customer_invoices.InvoiceNo.quantile(q=0.8)

In [91]:
germany_top = customer_invoices.query('InvoiceNo > @quantile_80').CustomerID

### 6) ⭐️Задание со звёздочкой!⭐️
Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

In [98]:
top_retail_germany = retail.query('CustomerID in @germany_top')

### 7) Сгруппируйте top_retail_germany по коду товара (StockCode). Какой из продуктов добавляли в корзину чаще всего, кроме POST? 
Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [104]:
top_retail_germany.query('StockCode != "POST"').groupby('StockCode').agg({'InvoiceNo':'nunique'}).idxmax()

InvoiceNo    22326
dtype: object

### 8) Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [106]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

In [107]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


### 9) Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

Например, для следующего примера...
```
InvoiceNo StockCode Quantity InvoiceDate          UnitPrice  CustomerID  Revenue
536365    85123A    6        2010-12-01 08:26:00  2.55       17850.0     15.30
536365    71053     6        2010-12-01 08:26:00  3.39       17850.0     20.34
555555    71053     1        2010-12-02 18:00:00  3.39       17850.0     3.39
...сумма заказа будет равна:

InvoiceNo  TotalRevenue
536365     35.64
555555     3.39
```
И ответом будет:
536365, 555555

In [117]:
sorted_revenues = (retail
                        .groupby('InvoiceNo', as_index = False)
                        .agg({'Revenue':'sum'})
                        .sort_values('Revenue', ascending = False)
                        .head(5)
)

In [121]:
print(sorted_revenues.InvoiceNo.str.cat(sep = ', '))

581483, 541431, 574941, 576365, 556444


# That's all!!!